<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [4]</a>'.</span>

# Preprocess and Sanity Check

In [5]:
%cd ../
%config InlineBackend.figure_format='retina'

import logging
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from IPython.core.getipython import get_ipython
from src.canonical_analysis.subspace_comm import CCARegions
from src.spikeloader import SpikeLoader

sns.set()
get_ipython().run_line_magic("config", "InlineBackend.figure_format='retina'")

/groups/stringer/home/josephs2/janelia2020


In [6]:
# logging.getLogger().setLevel(logging.INFO)
path_npz = '/groups/stringer/home/josephs2/data/text32_500_TX59_2020_08_18_coding_neurons.npz'
path_img = '/groups/pachitariu/pachitariulab/data/STIM/text32_500.mat'

In [7]:
# Parameters
path_npz = (
    "/groups/pachitariu/pachitariulab/datasets/v1RF/text32_500_TX59_2020_08_05.npz"
)
path_hdf5 = "/groups/stringer/home/josephs2/data/text32_500_TX59_2020_08_05.hdf5"
path_img = "/groups/pachitariu/pachitariulab/data/STIM/text32_500.mat"
path_loader = "/groups/stringer/home/josephs2/data/text32_500_TX59_2020_08_05.hdf5"
path_rf = "/groups/stringer/home/josephs2/data/text32_500_TX59_2020_08_05.hdf5"
path_gabor = "/groups/stringer/home/josephs2/data/text32_500_TX59_2020_08_05.hdf5"


In [8]:
import numpy as np
dat = np.load(path_npz)

In [10]:
for k in dat.keys():
    print(k, dat[k].shape)

istim (14838,)
spks (50559, 31904)
frame_start (14838,)
ypos (50559,)
xpos (50559,)
fspont (2228,)


In [17]:
dat_coding = np.load('/groups/stringer/home/josephs2/data/text32_500_TX59_2020_08_18_coding_neurons_TRUE.npy',
                    allow_pickle=True)
for k in dat_coding[].keys():
    print(k, dat_coding[k].shape)

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [12]:
dat['istim']

array([15734, 14602,  1285, ...,  4027, 15153, 13712], dtype=uint16)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [4]:
load_hdf5=True
out_file=path_hdf5
if Path(Path(out_file).with_suffix(".hdf5")).exists() and load_hdf5:
    logging.info("HDF5 exists, loading.")
    loader = SpikeLoader.from_hdf5(out_file, path_img)
else:
    logging.info("Reading from npz.")
    loader = SpikeLoader.from_npz(path_npz, path_img)
    logging.info("Saving to HDF5.")
    loader.save(out_file, overwrite=True)

spks shape:  (31904, 50559)
(31904, 50559)
Int64Index([    2,     4,     6,     8,    10,    12,    14,    16,    18,
               20,
            ...
            31884, 31886, 31888, 31890, 31892, 31894, 31896, 31898, 31900,
            31902],
           dtype='int64', length=14838)


IndexError: index 65535 is out of bounds for axis 0 with size 16000

In [ ]:
fig, ax = plt.subplots()
ax.scatter(loader.pos.x, loader.pos.y, s=0.5, alpha=0.2)
ax.set_aspect("equal")

In [ ]:
try:
    print(f"(Stim x Neu): {loader.spks.shape}")
    print(f"Number of spont frames: {loader.idx_spont.size}")
    print(f"Repeated stims: {loader.get_idx_rep().shape}")
except:
    pass 

### Correlations between the spiking responses of repeated and non-repeated stimuli.

In [ ]:
def corr_check(idx1, idx2, name):
    corr = CCARegions.pairwise_inner_prod(loader.S[idx1, :], loader.S[idx2, :], normalize=True)
    ax = sns.distplot(corr, label=name)
    ax.set_title("Correlations between stim")
    ax.set_xlabel("Pearson's $r$")
    return ax


rep = loader.get_idx_rep()
corr_check(rep[:1000, 0], rep[:1000, 1], "Repeat")
corr_check(np.arange(1000), np.arange(1000, 2000), "Non-repeat")
plt.legend()